In [ ]:
import tensorflow_datasets as tfds

In [2]:
import tensorflow as tf

In [31]:
import tensorflow_text as text

In [12]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True, as_supervised=True)

In [13]:
metadata.features, metadata.splits

(Translation({
     'en': Text(shape=(), dtype=string),
     'pt': Text(shape=(), dtype=string),
 }),
 {'train': <SplitInfo num_examples=51785, num_shards=1>,
  'validation': <SplitInfo num_examples=1193, num_shards=1>,
  'test': <SplitInfo num_examples=1803, num_shards=1>})

In [21]:
train_examples, val_examples, test_examples = examples['train'], examples['validation'], examples["test"]

In [29]:
for pt_examples, en_examples in train_examples.batch(4).take(1):
    for pt, en in zip(pt_examples.numpy(), en_examples.numpy()):
        print(pt.decode(), "\n", en.decode(), "\n")

e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade . 
 and when you improve searchability , you actually take away the one advantage of print , which is serendipity . 

mas e se estes fatores fossem ativos ? 
 but what if it were active ? 

mas eles não tinham a curiosidade de me testar . 
 but they did n't test for curiosity . 

e esta rebeldia consciente é a razão pela qual eu , como agnóstica , posso ainda ter fé . 
 and this conscious defiance is why i , as an agnostic , can still have faith . 



In [ ]:
tf.keras.utils.get_file(
    'ted_hrlr_translate_pt_en_converter.zip',
    'https://storage.googleapis.com/download.tensorflow.org/models/ted_hrlr_translate_pt_en_converter.zip',
    cache_dir='.', cache_subdir='', extract=True
)

184801/184801 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step


'./ted_hrlr_translate_pt_en_converter_extracted'

In [19]:
model_path = "/home/fs/ML/Ipynbs/DL/ted_hrlr_translate_pt_en_converter_extracted/ted_hrlr_translate_pt_en_converter"

In [20]:
tokenizers = tf.saved_model.load(model_path)

In [49]:
for _, en_examples in train_examples.batch(3).take(1):
    encoded = tokenizers.en.tokenize(en_examples)
    decoded = tokenizers.en.detokenize(encoded)

    for enc, dec in zip(encoded.numpy(), decoded.numpy()):
        print(enc, "\n", dec.decode(), "\n")

[   2   72  117   79 1259 1491 2362   13   79  150  184  311   71  103
 2308   74 2679   13  148   80   55 4840 1434 2423  540   15    3] 
 and when you improve searchability , you actually take away the one advantage of print , which is serendipity . 

[   2   87   90  107   76  129 1852   30    3] 
 but what if it were active ? 

[   2   87   83  149   50    9   56  664   85 2512   15    3] 
 but they did n ' t test for curiosity . 

